In [8]:
# Import necessary modules
from datetime import datetime  # Import datetime module to work with date and time
import uuid  # Import uuid module to generate unique identifiers

# Define a File class
class File:
    # Initialize the File object with specified attributes
    def __init__(self, name, file_type, size, blocks, protection):
        self.name = name  # Assign the file name
        self.file_type = file_type  # Assign the file type
        self.size = size  # Assign the file size
        self.blocks = blocks  # Assign the number of blocks
        self.protection = protection  # Assign the protection level
        self.creation_time = datetime.now()  # Set the creation time to the current date and time
        self.modification_time = None  # Initialize modification time as None
        self.access_time = None  # Initialize access time as None
        self.content = ""  # Initialize content as an empty string
        self.identifier = uuid.uuid4()  # Generate a unique identifier for the file
        self.parent_directory = None  # Initialize the parent directory as None

# Method to read the content of the file
    def read(self):
        self.access_time = datetime.now()  # Update the access time to the current date and time
        return self.content  # Return the content of the file

# Method to write additional content to the file
    def write(self):
        # Prompt user for additional content
        additional_content = input(f"Enter additional content for file '{self.name}': ")
        original_size = len(self.content)  # Store the original size of the file content
        # Append additional content to the existing content
        self.content += additional_content
        # Calculate the size difference and update the size of the file
        size_difference = len(self.content) - original_size
        self.size += size_difference
        # Update modification time to the current date and time
        self.modification_time = datetime.now()
        # Update the size of the parent directory
        if self.parent_directory:
            self.parent_directory.update_directory_space(size_difference)

# Method to copy the file to a new destination directory
# Parameters: - destination_directory (Directory): The destination directory where the file will be copied.
#  This method creates a new file with a modified name (appending ".c") and copies the content from the original file.
#  The new file is then added to the specified destination directory
    def copy(self, destination_directory):
        # Create a new file with a modified name and copy content
        new_file = File(self.name+".c", self.file_type, self.size, self.blocks, self.protection)
        new_file.content = self.content
        # Add the new file to the destination directory
        destination_directory.add_child(new_file)
        print(f"File '{self.name}' copied to directory '{destination_directory.name}' successfully.\n")

# Method to move the file to a new location (directory)
#Parameters: - new_location (Directory): The destination directory to which the file will be moved.
#If the file has a current parent directory, it is first removed from that directory, and then added to the specified new location
    def move(self, new_location): 
        if self.parent_directory:
            # Remove the file from the current parent directory
            self.parent_directory.remove_child(self)
            # Add the file to the new location (directory)
            new_location.add_child(self)
            print(f"File '{self.name}' moved to directory '{new_location.name}' successfully.\n")
        else:
            print("File has no parent directory. Unable to move.")
            
# Method to delete the file
# Parameters: - parent_directory (Directory, optional): The parent directory from which the file will be deleted.
    def delete(self, parent_directory=None):
        if parent_directory:
            # Remove the file from the specified parent directory
            parent_directory.remove_child(self)
            print(f"File '{self.name}' deleted successfully from directory '{parent_directory.name}'.\n")
        elif self.parent_directory:
            # Prompt user for the name of the parent directory to delete the file
            parent_name = input(f"Enter the name of the parent directory to delete file '{self.name}': ")
            if parent_name == self.parent_directory.name:
                # Remove the file from its parent directory
                self.parent_directory.remove_child(self)
                print(f"File '{self.name}' deleted successfully.\n")
            else:
                print(f"Parent directory '{parent_name}' does not match. Unable to delete file.\n")
        else:
            print("File has no parent directory. Unable to delete.")

# Method to rename the file
# Parameters: - new_name (str): The new name for the file.
#             - parent_directory (Directory, optional): The parent directory within which the file will be renamed
    def rename(self, new_name, parent_directory=None):
        if parent_directory:
            # Rename the file within the specified parent directory
            original_name = self.name
            self.name = new_name
            print(f"File '{original_name}' renamed to '{new_name}' in directory '{parent_directory.name}' successfully.\n")
        elif self.parent_directory:
            # Prompt user for the name of the parent directory to rename the file
            parent_name = input(f"Enter the name of the parent directory to rename file '{self.name}': ")
            if parent_name == self.parent_directory.name:
                # Rename the file within its parent directory
                original_name = self.name
                self.name = new_name
                print(f"File '{original_name}' renamed to '{new_name}' successfully.\n")
            else:
                print(f"Parent directory '{parent_name}' does not match. Unable to rename file.\n")
        else:
            print("File has no parent directory. Unable to rename.")

# Method to get information about the file
    def get_info(self):
        return {
            'Name': self.name,
            'Type': self.file_type,
            'Size': f'{self.size} bytes',
            'Blocks': self.blocks,
            'Protection': self.protection,
            'Identifier': str(self.identifier),
            'Creation Time': self.creation_time,
            'Modification Time': self.modification_time,
            'Access Time': self.access_time,
            'Content': self.content
        }
# Define a Directory class
class Directory:
    # Initialize the Directory object with a name
    def __init__(self, name):
        self.name = name
        self.children = []  # List to store children (files or subdirectories)
        self.parent_directory = None  # Reference to the parent directory
        self.size = 0  # Total size of the directory

# Method to add a child (file or subdirectory) to the directory
    def add_child(self, child):
        self.children.append(child)  # Add child to the list
        child.parent_directory = self  # Set the child's parent directory to this directory
        self.update_directory_space(child.size)  # Update the total size of the directory

# Method to remove a child (file or subdirectory) from the directory
    def remove_child(self, child):
        if child in self.children:  # Check if the child is present in the directory
            self.update_directory_space(-child.size)  # Update the total size of the directory
            self.children.remove(child)  # Remove the child from the list
        else:
            print(f"Child '{child.name}' not found in directory '{self.name}'. Unable to remove.")

# Method to create a file within the directory
#This method creates a new File object with the specified attributes and content.
#The new file is added as a child of the directory using the 'add_child' method.
#The created file object is then returned.
    def create_file(self, name, file_type, content, blocks, protection):
        file = File(name, file_type, len(content), blocks, protection)  # Create a new File object
        file.content = content  # Set the content of the file
        self.add_child(file)  # Add the file as a child of the directory
        return file  # Return the created file

# Method to create a subdirectory within the directory
    def create_directory(self, name):
        new_directory = Directory(name)  # Create a new Directory object
        self.add_child(new_directory)  # Add the new directory as a child
        print(f"Directory '{new_directory.name}' created successfully. Size: {new_directory.size} bytes.\n")
        return new_directory  # Return the created directory

# Method to recursively find a child by name within the directory
    def find_child_recursive(self, target_name):
        if self.name == target_name:
            return self  # Return the current directory if the name matches

        for child in self.children:
            if isinstance(child, Directory):
                result = child.find_child_recursive(target_name)  # Recursively search in subdirectories
                if result:
                    return result
            elif isinstance(child, File) and child.name == target_name:
                return child  # Return the file if the name matches

        return None  # Return None if the child is not found

 # Method to delete the directory
    def delete(self, parent_directory=None):
        if parent_directory:
            parent_directory.remove_child(self)  # Remove the directory from the specified parent directory
            print(f"Directory '{self.name}' deleted successfully from directory '{parent_directory.name}'.\n")
        elif self.parent_directory:
            parent_name = input(f"Enter the name of the parent directory to delete directory '{self.name}': ")
            if parent_name == self.parent_directory.name:
                self.parent_directory.remove_child(self)  # Remove the directory from its parent directory
                print(f"Directory '{self.name}' deleted successfully.\n")
            else:
                print(f"Parent directory '{parent_name}' does not match. Unable to delete directory.\n")
        else:
            print("Directory has no parent. Unable to delete.")

# Method to rename the directory
    def rename(self, new_name, parent_directory=None):
        if parent_directory:
            original_name = self.name
            self.name = new_name
            print(f"Directory '{original_name}' renamed to '{new_name}' in directory '{parent_directory.name}' successfully.\n")
        elif self.parent_directory:
            parent_name = input(f"Enter the name of the parent directory to rename directory '{self.name}': ")
            if parent_name == self.parent_directory.name:
                original_name = self.name
                self.name = new_name
                print(f"Directory '{original_name}' renamed to '{new_name}' successfully.\n")
            else:
                print(f"Parent directory '{parent_name}' does not match. Unable to rename directory.\n")
        else:
            print("Directory has no parent. Unable to rename.")

# Method to list the contents (files and subdirectories) of the directory
    def list_contents(self):
        contents = []
        for child in self.children:
            contents.append(f'{child.__class__.__name__}: {child.name}')
        return contents
    
# Method to get information about the directory
    def get_info(self):
        return {
            'Name': self.name,
            'Contents': self.list_contents()
        }

# Method to update the total size of the directory and its ancestors
    def update_directory_space(self, size):
        self.size += size  # Update the total size of the directory
        if self.parent_directory:
            self.parent_directory.update_directory_space(size)  # Recursively update the size of parent directories

# Define a Partition class
class Partition:
    # Initialize the Partition object with specified attributes
    def __init__(self, label, size, block_size):
        self.label = label  # Assign the partition label
        self.uuid = uuid.uuid4()  # Generate a unique identifier for the partition
        self.size = size  # Assign the partition size
        self.used_space = 0  # Initialize used space as 0
        self.free_space = size  # Initialize free space as the total size
        self.block_size = block_size  # Assign the block size
        # Create a root directory for the partition
        self.root_directory = Directory('root')
        self.root_directory.parent_directory = self  # Set the parent directory for the root

    # Method to update the used and free space of the partition
    def update_directory_space(self, size):
        self.used_space += size
        self.free_space -= size

    # Method to get information about the partition
    def get_info(self):
        return {
            'Label': self.label,
            'UUID': str(self.uuid),
            'Size': f'{self.size} bytes',
            'Used Space': f'{self.used_space} bytes',
            'Free Space': f'{self.free_space} bytes',
            'Block Size': f'{self.block_size} bytes',
            'Root Directory': self.root_directory.get_info()
        }

# Function to write content to a file
def write_to_file(file):
    file.write()
    print(f"Content written to file '{file.name}' successfully. File size: {file.size} bytes.\n")

# Function to read content from a file
def read_from_file(file):
    content = file.read()
    print(f"Content of file '{file.name}':\n{content}\n")

# Function to copy a file or directory
def copy_file(file, destination_directory):
    file.copy(destination_directory)

# Function to move a file or directory
def move_file(file, destination_directory):
    file.move(destination_directory)

# Function to rename a file or directory
def rename_file_or_directory(item, new_name, parent_directory=None):
    item.rename(new_name, parent_directory)

# Function to display the directory tree
def display_tree(node, indent=0):
    if isinstance(node, Directory):
        print("  " * indent + f"- {node.name} (Directory)")
        for child in node.children:
            display_tree(child, indent + 1)
    elif isinstance(node, File):
        print("  " * indent + f"- {node.name} (File)")



# Main function to interact with the partition and perform operations
#     The main function that serves as the entry point for the file system simulation program.
#     It creates a partition with a root directory and enters a loop to continuously prompt the user for operations.
#     The available operations include creating files and directories, reading and writing to files, copying, moving, renaming, and deleting files and directories. The user can also exit the program.
#     The user is presented with the current partition information and directory tree on each iteration.
#     Operations:
     - 'f': Create a file
     - 'd': Create a directory
     - 'w': Write to a file
     - 'r': Read from a file
     - 'cp': Copy a file or directory
     - 'mv': Move a file or directory
     - 'rn': Rename a file or directory
     - 'del': Delete a file or directory
     - 'exit': Exit the program
     The loop continues until the user chooses to exit.
# """

In [9]:

def main():
    partition = Partition(label='C:', size=1024, block_size=4)

    while True:
        print("\nPartition Information:")
        print(partition.get_info())

        # Display the directory tree
        print("\nDirectory Tree:")
        display_tree(partition.root_directory)

        # Prompt user for the operation to perform
        operation = input(
            "Do you want to create a file (f), create a directory (d), write to a file (w), read from a file (r), "
            "copy a file or directory (cp), move a file or directory (mv), rename a file or directory (rn), delete a file or directory (del), or exit (exit)? ").lower()

        if operation == 'f':
            file_name = input("Enter the name of the file: ")
            file_content = input("Enter the content of the file: ")
            partition.root_directory.create_file(file_name, 'Text File', file_content, 1, 'rw-')

        elif operation == 'd':
            directory_name = input("Enter the name of the directory: ")
            new_directory = partition.root_directory.create_directory(directory_name)

        elif operation == 'w':
            file_name = input("Enter the name of the file you want to write to: ")
            file = partition.root_directory.find_child_recursive(file_name)
            if isinstance(file, File):
                write_to_file(file)
            else:
                print(f"File '{file_name}' not found.")

        elif operation == 'r':
            file_name = input("Enter the name of the file you want to read: ")
            file = partition.root_directory.find_child_recursive(file_name)
            if isinstance(file, File):
                read_from_file(file)
            else:
                print(f"File '{file_name}' not found.")

        elif operation == 'cp':
            source_name = input("Enter the name of the file or directory you want to copy: ")
            destination_name = input("Enter the name of the destination directory or partition: ")
            source = partition.root_directory.find_child_recursive(source_name)
            destination = partition.root_directory.find_child_recursive(destination_name)

            if source and destination and isinstance(source, (File, Directory)):
                if isinstance(destination, Directory):
                    copy_file(source, destination)
                else:
                    print("Destination must be a directory.")
            else:
                print(f"Source '{source_name}' or destination '{destination_name}' not found.")

        elif operation == 'mv':
            source_name = input("Enter the name of the file or directory you want to move: ")
            destination_name = input("Enter the name of the destination directory or partition: ")
            source = partition.root_directory.find_child_recursive(source_name)
            destination = partition.root_directory.find_child_recursive(destination_name)

            if source and destination and isinstance(source, (File, Directory)):
                if isinstance(destination, Directory):
                    move_file(source, destination)
                else:
                    print("Destination must be a directory.")
            else:
                print(f"Source '{source_name}' or destination '{destination_name}' not found.")

        elif operation == 'rn':
            item_name = input("Enter the name of the file or directory you want to rename: ")
            new_name = input("Enter the new name: ")
            item = partition.root_directory.find_child_recursive(item_name)
            if item and isinstance(item, (File, Directory)):
                item_parent_name = input(f"Enter the name of the parent directory to rename '{item_name}': ")
                rename_file_or_directory(item, new_name, partition.root_directory.find_child_recursive(item_parent_name))
            else:
                print(f"Item '{item_name}' not found or is not a file or directory.")

        elif operation == 'del':
            item_name = input("Enter the name of the file or directory you want to delete: ")
            item = partition.root_directory.find_child_recursive(item_name)
            if item and isinstance(item, (File, Directory)):
                item_parent_name = input(f"Enter the name of the parent node to delete '{item_name}': ")
                item.delete(parent_directory=partition.root_directory.find_child_recursive(item_parent_name))
            else:
                print(f"Item '{item_name}' not found or is not a file or directory.")

        elif operation == 'exit':
            break

        else:
            print("Invalid operation. Please try again.")

# Execute the main function if the script is run as the main module
if __name__ == "__main__":
    main()



Partition Information:
{'Label': 'C:', 'UUID': '7c67ec70-415d-4f14-b308-ea75999b268b', 'Size': '1024 bytes', 'Used Space': '0 bytes', 'Free Space': '1024 bytes', 'Block Size': '4 bytes', 'Root Directory': {'Name': 'root', 'Contents': []}}

Directory Tree:
- root (Directory)
Do you want to create a file (f), create a directory (d), write to a file (w), read from a file (r), copy a file or directory (cp), move a file or directory (mv), rename a file or directory (rn), delete a file or directory (del), or exit (exit)? d
Enter the name of the directory: e
Directory 'e' created successfully. Size: 0 bytes.


Partition Information:
{'Label': 'C:', 'UUID': '7c67ec70-415d-4f14-b308-ea75999b268b', 'Size': '1024 bytes', 'Used Space': '0 bytes', 'Free Space': '1024 bytes', 'Block Size': '4 bytes', 'Root Directory': {'Name': 'root', 'Contents': ['Directory: e']}}

Directory Tree:
- root (Directory)
  - e (Directory)
Do you want to create a file (f), create a directory (d), write to a file (w), re